**Table of contents**<a id='toc0_'></a>    
- 1. [Imports  ](#toc1_)    
- 2. [Einstellungen  ](#toc2_)    
- 3. [Filial-Steckbrief  ](#toc3_)    
- 4. [Wöchentliche Umsätze, Kunden und pro-Kopf-Umsätze ](#toc4_)    
- 5. [Promo  ](#toc5_)    
- 6. [StateHoliday  ](#toc6_)    
  - 6.1. [Verteilung der Feiertage und ggf. Öffnungen an Feiertagen ](#toc6_1_)    
  - 6.2. [Umsätze und Kundenzahlen und deren Verteilung an Feiertagen ](#toc6_2_)    
- 7. [SchoolHoliday  ](#toc7_)    
  - 7.1. [Verteilung der Ferientage ](#toc7_1_)    
  - 7.2. [Wöchentliche Umsätze nach Ferientagen ](#toc7_2_)    
  - 7.3. [Umsätze und Kundenzahlen und deren Verteilung an Ferientagen ](#toc7_3_)    
- 8. [Wöchentliche und jahresweise Saisonalität der Umsätze und Kundenzahlen  ](#toc8_)    
- 9. [Weitere Metriken  ](#toc9_)    
  - 9.1. [CumSales  ](#toc9_1_)    
  - 9.2. [DiffSales  ](#toc9_2_)    
  - 9.3. [Percent Changes  ](#toc9_3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
store_id = 123

In [2]:
# %pip install plotly

# 1. <a id='toc1_'></a>Imports   [&#8593;](#toc0_)

In [3]:
import numpy as np
import pandas as pd

import pms
import rms

import warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from plotly.subplots import make_subplots

from IPython.display import display
from IPython.display import HTML

# 2. <a id='toc2_'></a>Einstellungen   [&#8593;](#toc0_)

In [4]:
# falls Jupyter bei der Ausgabe von dfs Spalten abgeschneidet
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

# damit plotly mit Jupyter funktioniert
#pio.templates['plotly'].layout['autosize'] = False
# pio.templates.default = "presentation"


# Errors should never pass silently. Unless explicitly silenced.
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
data = rms.get_store_data_df(store_id)
data_open = data[data.Open==1]
weekly_data = rms.get_weekly_data(store_id)

# 3. <a id='toc3_'></a>Filial-Steckbrief   [&#8593;](#toc0_)

In [5]:
df = pms.print_store_info(store_id)

***** General Information about Store 123 ***** 

Sales total..............: 6,450,589 EUR
Sales total rank.........: #232 / 1115 stores 

Sales per open day.......: 8,399.20 EUR
Sales per open day rank..: #223 / 1115 stores 

Customers total..........: 737,109
Customers total rank.....: #197 / 1115 stores 

Customers per open day...: 960
Customers per open day rank: #193 / 1115 stores 

Sales per Customer.......: 8.751 EUR
Sales per Customer rank..: #706 / 1115 stores 

Days open................: 768 / 942 days
Closed for renovation....: no

Promodays................: 360 / 942 days 

Open on State Holidays...: 0 / 29 days

School Holidays..........: 168 

Store Type...............: a
Assortment...............: basic 

Competition Distance.....: 16,760m
Competition Since........: 2011-09-01 00:00:00 

Promo2...................: no


# 4. <a id='toc4_'></a>Wöchentliche Umsätze, Kunden und pro-Kopf-Umsätze  [&#8593;](#toc0_)

Mit ``pms.keep_rolling(data, col, windows_list)`` kann man eine Spalte mit moving averages plotten und auf Trends untersuchen. Wenn CompetitionSince oder Promo2Since im Beobachtungszeitraum liegen, werden die Zeitpunkte im Diagramm markiert.

In [7]:
pms.keep_rolling(weekly_data, "Sales", [4, 12, 52])

In [8]:
pms.keep_rolling(weekly_data, "Customers", [4, 12, 26])

In [9]:
pms.keep_rolling(weekly_data, "spc", [4, 12, 26, 52])

# 5. <a id='toc5_'></a>Promo   [&#8593;](#toc0_)

Da Promo immer von Montag bis Freitag stattfindet, vergleichen wir auch nur diese Zeiträume für Promo und Nicht-Promowochen.

In [25]:
mon_fr_sales = (data[data.DayOfWeek<=5]
                .loc[:,["Sales", "Customers", "Promo"]]
                .resample('W')
                .sum())

mon_fr_sales.Promo = mon_fr_sales.Promo.astype("category")
mon_fr_sales.loc[:, "spc"] = mon_fr_sales.Sales / mon_fr_sales.Customers

px.scatter(mon_fr_sales, x=mon_fr_sales.index, y="Sales", color="Promo", hover_data=["Customers", "spc"], title="Weekly Sales (mon-fr) by Promo-weeks and non-Promo-weeks")

In [11]:
px.bar(mon_fr_sales.groupby("Promo")[["Sales", "Customers"]].mean(), barmode="group", width=1100, title="Sales, Customers (mon-fr, mean) by Promo")

In [12]:
pms.plotly_boxes(mon_fr_sales, "Promo")

# 6. <a id='toc6_'></a>StateHoliday   [&#8593;](#toc0_)
## 6.1. <a id='toc6_1_'></a>Verteilung der Feiertage und ggf. Öffnungen an Feiertagen  [&#8593;](#toc0_)

In [13]:
px.scatter(data, x=data.index, y="StateHoliday", hover_data=["Open"], color="Open", category_orders={"StateHoliday": ["c", "b", "a", "0"]}, width=1100, title=f"StateHolidays for store {data.Store.iloc[0]}")


## 6.2. <a id='toc6_2_'></a>Umsätze und Kundenzahlen und deren Verteilung an Feiertagen  [&#8593;](#toc0_)

In [14]:
pms.plotly_boxes(data, "StateHoliday")

# 7. <a id='toc7_'></a>SchoolHoliday   [&#8593;](#toc0_)

## 7.1. <a id='toc7_1_'></a>Verteilung der Ferientage  [&#8593;](#toc0_)

In [15]:
px.scatter(data.SchoolHoliday, x=data.SchoolHoliday.index, y=data.SchoolHoliday.values, width=1100, title=f"SchoolHolidays for store {store_id}")


## 7.2. <a id='toc7_2_'></a>Wöchentliche Umsätze nach Ferientagen  [&#8593;](#toc0_)

In [16]:
weekly_data.SchoolHoliday = weekly_data.SchoolHoliday.astype("category")

px.scatter(weekly_data, x=weekly_data.index, y="Sales", color="SchoolHoliday", category_orders={"SchoolHoliday": range(6)}, width=1100, 
           title=f"Weekly Sales by SchoolHolidays for store {store_id}", color_discrete_sequence=px.colors.qualitative.T10)


## 7.3. <a id='toc7_3_'></a>Umsätze und Kundenzahlen und deren Verteilung an Ferientagen  [&#8593;](#toc0_)

In [17]:
pms.plotly_boxes(data_open, "SchoolHoliday")

# 8. <a id='toc8_'></a>Wöchentliche und jahresweise Saisonalität der Umsätze und Kundenzahlen   [&#8593;](#toc0_)

In [18]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)
fig.add_trace(go.Box(x=data_open.DayOfWeek, y=data_open.Sales, boxpoints="all"), row=1, col=1)
fig.add_trace(go.Box(x=data_open.DayOfWeek, y=data_open.Customers, boxpoints="all" ), row=2, col=1)
fig.update_layout(height=800, width=1100, title_text=f"Weekly Seasonality of Sales and Customers for store {store_id} ", showlegend=False)
fig.update_xaxes(title_text="DayOfWeek", row=2, col=1)
fig.update_yaxes(title_text="Sales", row=1, col=1)
fig.update_yaxes(title_text="Customers", row=2, col=1)
fig.show()

In [19]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Box(x=data_open.index.month, y=data_open.Sales, boxpoints="all"), row=1, col=1)
fig.add_trace(go.Box(x=data_open.index.month, y=data_open.Customers, boxpoints="all" ), row=2, col=1)
fig.update_layout(height=800, width=1100, title_text=f"Yearly Seasonality of Sales and Customers for store {store_id}", showlegend=False)
fig.update_xaxes(title_text="Month", row=2, col=1)
fig.update_yaxes(title_text="Sales", row=1, col=1)
fig.update_yaxes(title_text="Customers", row=2, col=1)
fig.show()


# 9. <a id='toc9_'></a>Weitere Metriken   [&#8593;](#toc0_)

## 9.1. <a id='toc9_1_'></a>CumSales   [&#8593;](#toc0_)

In [20]:
px.line(weekly_data.cum_sales, title=f"Cummulative Sales for Store {store_id}")

## 9.2. <a id='toc9_2_'></a>DiffSales   [&#8593;](#toc0_)

In [21]:
px.line(data.diff_sales, title=f"Daily Sales Changes for Store {store_id}")

In [22]:
px.line(weekly_data.diff_sales, title=f"Weekly Sales Changes for Store {store_id}")

## 9.3. <a id='toc9_3_'></a>Percent Changes   [&#8593;](#toc0_)

In [23]:
px.line(data_open.pct_change_sales, title=f"Daily Sales Percent Changes for Store {store_id}")

In [24]:
px.line(weekly_data.pct_change_sales, title=f"Weekly Sales Percent Changes for Store {store_id}")